In [38]:
from pyspark.sql import SparkSession
from os.path import abspath
from pyspark.sql import functions as F


In [54]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Exercicio Intro") \
    .getOrCreate()

caminho_absoluto = abspath("nomes_aleatorios.txt")
df_nomes = spark.read.csv(caminho_absoluto, inferSchema=True)

df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



In [55]:
df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")
df_nomes.show(10)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



In [56]:
valores_escolaridade = ["Fundamental", "Médio", "Superior"]

df_nomes = df_nomes.withColumn("Escolaridade", 
                                F.when(F.rand() < 1/3, valores_escolaridade[0])
                                .when(F.rand() < 2/3, valores_escolaridade[1])
                                .otherwise(valores_escolaridade[2]))

df_nomes.show(10)

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|   Frances Bennet| Fundamental|
|    Jamie Russell| Fundamental|
|   Edward Kistler| Fundamental|
|    Sheila Maurer| Fundamental|
| Donald Golightly|    Superior|
|       David Gray|       Médio|
|      Joy Bennett| Fundamental|
|      Paul Kriese|    Superior|
|Berniece Ornellas|       Médio|
|    Brian Farrell|       Médio|
+-----------------+------------+
only showing top 10 rows



In [58]:
paises_sulamericanos = [
    "Brasil", "Argentina", "Colômbia", "Venezuela", "Peru", 
    "Chile", "Equador", "Bolívia", "Paraguai", "Uruguai", 
    "Guiana", "Suriname", "Guiana Francesa"
]
# Adicionando a coluna "Pais" com valores aleatórios
df_nomes = df_nomes.withColumn("Pais",
    F.element_at(
        F.array([F.lit(p) for p in paises_sulamericanos]),
        (F.floor(F.rand() * F.size(F.array([F.lit(p) for p in paises_sulamericanos]))) + 1).cast("int")
    )
)
df_nomes.show(10)



+-----------------+------------+---------+
|            Nomes|Escolaridade|     Pais|
+-----------------+------------+---------+
|   Frances Bennet| Fundamental|  Uruguai|
|    Jamie Russell| Fundamental|   Brasil|
|   Edward Kistler| Fundamental| Suriname|
|    Sheila Maurer| Fundamental| Colômbia|
| Donald Golightly|    Superior|     Peru|
|       David Gray|       Médio|   Guiana|
|      Joy Bennett| Fundamental| Paraguai|
|      Paul Kriese|    Superior|   Brasil|
|Berniece Ornellas|       Médio|  Bolívia|
|    Brian Farrell|       Médio|Argentina|
+-----------------+------------+---------+
only showing top 10 rows



In [59]:
df_nomes = df_nomes.withColumn("AnoNascimento", (F.rand() * (2010 - 1945) + 1945).cast("int"))

df_nomes.show(10)

+-----------------+------------+---------+-------------+
|            Nomes|Escolaridade|     Pais|AnoNascimento|
+-----------------+------------+---------+-------------+
|   Frances Bennet| Fundamental|  Uruguai|         1946|
|    Jamie Russell| Fundamental|   Brasil|         1949|
|   Edward Kistler| Fundamental| Suriname|         1972|
|    Sheila Maurer| Fundamental| Colômbia|         1958|
| Donald Golightly|    Superior|     Peru|         1993|
|       David Gray|       Médio|   Guiana|         1989|
|      Joy Bennett| Fundamental| Paraguai|         1972|
|      Paul Kriese|    Superior|   Brasil|         1980|
|Berniece Ornellas|       Médio|  Bolívia|         1978|
|    Brian Farrell|       Médio|Argentina|         1945|
+-----------------+------------+---------+-------------+
only showing top 10 rows



In [60]:
df_select = df_nomes.filter(F.col("AnoNascimento") >= 2000)

df_select.show(10)

+--------------------+------------+---------------+-------------+
|               Nomes|Escolaridade|           Pais|AnoNascimento|
+--------------------+------------+---------------+-------------+
|        James Barton|    Superior|           Peru|         2006|
|Christopher Williams| Fundamental|         Guiana|         2009|
|       Jenna Johnson|       Médio|           Peru|         2007|
|     Katherine Moore| Fundamental|         Brasil|         2009|
|        Jerry Remick| Fundamental|         Brasil|         2001|
|      Michael Farmer| Fundamental|         Brasil|         2001|
|         John Snipes| Fundamental|       Paraguai|         2004|
|      Diane Spillett| Fundamental|Guiana Francesa|         2009|
|       Fannie Taylor|    Superior|           Peru|         2008|
|      Bradley Nester|       Médio|        Uruguai|         2008|
+--------------------+------------+---------------+-------------+
only showing top 10 rows



In [61]:
df_nomes.createOrReplaceTempView("pessoas")

df_select_sql = spark.sql("SELECT * FROM pessoas WHERE AnoNascimento >= 2000")

df_select_sql.show(10)

+--------------------+------------+---------------+-------------+
|               Nomes|Escolaridade|           Pais|AnoNascimento|
+--------------------+------------+---------------+-------------+
|        James Barton|    Superior|           Peru|         2006|
|Christopher Williams| Fundamental|         Guiana|         2009|
|       Jenna Johnson|       Médio|           Peru|         2007|
|     Katherine Moore| Fundamental|         Brasil|         2009|
|        Jerry Remick| Fundamental|         Brasil|         2001|
|      Michael Farmer| Fundamental|         Brasil|         2001|
|         John Snipes| Fundamental|       Paraguai|         2004|
|      Diane Spillett| Fundamental|Guiana Francesa|         2009|
|       Fannie Taylor|    Superior|           Peru|         2008|
|      Bradley Nester|       Médio|        Uruguai|         2008|
+--------------------+------------+---------------+-------------+
only showing top 10 rows



In [32]:
df_millennials = df_nomes.filter((F.col("AnoNascimento") >= 1980) & (F.col("AnoNascimento") <= 1994))

count_millennials = df_millennials.count()

print(f"Número de pessoas da geração Millennials: {count_millennials}")

Número de pessoas da geração Millennials: 2307242


In [39]:
df_nomes.createOrReplaceTempView("pessoas")

count_millennials_sql = spark.sql("SELECT COUNT(*) FROM pessoas WHERE AnoNascimento BETWEEN 1980 AND 1994")

count_millennials_sql.show()

+--------+
|count(1)|
+--------+
| 2307242|
+--------+



In [53]:
df_nomes.createOrReplaceTempView("pessoas")
df_z = spark.sql("""
    SELECT Pais, COUNT(*) AS Quantidade
    FROM pessoas
    WHERE AnoNascimento BETWEEN 1995 AND 2015
    GROUP BY Pais
    ORDER BY Pais
""")

df_z.show()

+---------------+----------+
|           Pais|Quantidade|
+---------------+----------+
|      Argentina|    177498|
|        Bolívia|    177724|
|         Brasil|    177318|
|          Chile|    177402|
|       Colômbia|    177329|
|        Equador|    177070|
|         Guiana|    177881|
|Guiana Francesa|    177420|
|       Paraguai|    178016|
|           Peru|    177058|
|       Suriname|    177795|
|        Uruguai|    177265|
|      Venezuela|    177212|
+---------------+----------+



In [65]:
df_nomes.show(10)

+-----------------+------------+---------+-------------+
|            Nomes|Escolaridade|     Pais|AnoNascimento|
+-----------------+------------+---------+-------------+
|   Frances Bennet| Fundamental|  Uruguai|         1946|
|    Jamie Russell| Fundamental|   Brasil|         1949|
|   Edward Kistler| Fundamental| Suriname|         1972|
|    Sheila Maurer| Fundamental| Colômbia|         1958|
| Donald Golightly|    Superior|     Peru|         1993|
|       David Gray|       Médio|   Guiana|         1989|
|      Joy Bennett| Fundamental| Paraguai|         1972|
|      Paul Kriese|    Superior|   Brasil|         1980|
|Berniece Ornellas|       Médio|  Bolívia|         1978|
|    Brian Farrell|       Médio|Argentina|         1945|
+-----------------+------------+---------+-------------+
only showing top 10 rows



In [67]:
df_nomes.createOrReplaceTempView("tabela")

# Executar a consulta para encontrar os nomes únicos
nomes_unicos = spark.sql("""
    SELECT DISTINCT Nomes
    FROM tabela
""")

# Exibir os nomes únicos
nomes_unicos.show()

+----------------+
|           Nomes|
+----------------+
|      Frank Rose|
|   Scott Aguilar|
| Alfredo Schnell|
|   Sandra Brasil|
|   Leon Kanahele|
|      Eric Adams|
|       Ryan Dean|
|    Corey Keller|
|     Arthur Gumm|
|Walter Humpherys|
|Randall Grizzard|
|    Richard King|
|   Mary Williams|
|    Rebecca Gass|
|   Adam Williams|
|   Corey Mcaulay|
|    Roxie Bernal|
|    David Lowman|
|  Enrique Miller|
|  Jackie Leonard|
+----------------+
only showing top 20 rows

